In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from utils.path_utils import project_root
from utils.loader import initialize_experiment, make_loader, get_train_test_indicies, DatasetWithPadding

import os

from train_gtn import load_model, GatedTransformerNetwork, train_model
from utils.config import gtn_param

import torch
from torch.utils.data import DataLoader


In [2]:
training_examples = os.path.join(os.path.join(project_root(), 'data', 'processed', 'test_final_dataset.pickle'))
all_data = pd.read_pickle(training_examples)
data = pd.concat(all_data)

In [4]:
print(data.shape)
data.head()
data.columns

(1552210, 65)


Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel', 'PatientID', 'MAP_SOFA',
       'Bilirubin_total_SOFA', 'Platelets_SOFA', 'SOFA_score',
       'SOFA_score_diff', 'SOFA_deterioration', 'ResP_qSOFA', 'SBP_qSOFA',
       'qSOFA_score', 'qSOFA_score_diff', 'qSOFA_deterioration',
       'qSOFA_indicator', 'SOFA_indicator', 'Mortality_sofa', 'Temp_sirs',
       'HR_sirs', 'Resp_sirs', 'paco2_sirs', 'wbc_sirs', 'infection_proxy',
       't_suspicion', 't_sofa', 't_sepsis'],
      dtype='object')

In [5]:
sample_data = pd.read_csv(os.path.join(project_root(), 'physionet.org', 'files', 'challenge-2019', '1.0.0', 'training', 'training_setA', 'p000001.psv'), sep='|')
print(sample_data.shape)

sample_data.head()

(54, 41)


,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,...,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,2,0
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,...,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,3,0
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,...,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,4,0
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,...,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,5,0


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (40336,) + inhomogeneous part.

In [ ]:
# # Getting train and test indicies
# train_indicies, test_indicies = get_train_test_indicies()

# # training_examples, lengths_list, is_sepsis 
# training_examples, lengths_list, is_sepsis = initialize_experiment()

# # Train and test samples
# train_samples = [training_examples[idx] for idx in train_indicies]
# test_samples = [training_examples[idx] for idx in test_indicies]

# train_lengths_list = [lengths_list[idx] for idx in train_indicies]
# test_lengths_list = [lengths_list[idx] for idx in test_indicies]

# is_sepsis_train = [is_sepsis[idx] for idx in train_indicies]
# is_sepsis_test = [is_sepsis[idx] for idx in test_indicies]

# # Converting data to train and test sets
# train_dataset = DatasetWithPadding(training_examples_list=train_samples, lengths_list=train_lengths_list, is_sepsis=is_sepsis_train)
# test_dataset = DatasetWithPadding(training_examples_list=test_samples, lengths_list=test_lengths_list, is_sepsis=is_sepsis_test,)

In [ ]:
# # Collate Fn
# def collate_fn(batch):
    
#     # Sequeneces and Lengths
#     sequences, labels = zip(*batch)
#     lengths = torch.tensor([len(seq) for seq in sequences])

#     # Padding
#     sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    
#     # Creating masks
#     masks = torch.zeros(sequences_padded.size(0), sequences_padded.size(1), dtype=torch.bool)
#     for i, length in enumerate(lengths):
#         masks[i, :length] = 1

#     return sequences_padded, masks, torch.tensor(labels, dtype=torch.long)


# # Loaders
# train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True, num_workers=10, collate_fn=collate_fn)

# for inputs, masks, targets in train_loader:
#     print(inputs.shape, masks.shape, targets.shape)
#     break


In [ ]:
training_examples, lengths_list, is_sepsis = initialize_experiment()
train_loader, test_loader, train_indicies, test_indicies = make_loader(training_examples, lengths_list, is_sepsis, batch_size=3, mode="padding_masking")

In [ ]:
for inputs, labels, masks in train_loader:
    print(inputs.shape, labels.shape, masks.shape)
    break

In [ ]:
print(inputs[0])

In [ ]:
masks[0]

In [ ]:
score = torch.rand(336, 63)
score.masked_fill(~masks[0], float('-inf'))  # New

In [ ]:
masks.unsqueeze(1).expand(3, 336, 336)

In [ ]:
-

In [ ]:
-

In [ ]:
import numpy as np
import pandas as pd
import torch

from utils.evaluate_helper_methods import *
from utils.path_utils import project_root
from utils.evaluate_sepsis_score import evaluate_sepsis_score
from utils.get_true_labels import get_true_labels

from utils.helpers import get_features

import tqdm


def get_sepsis_score(data, model):
    columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp',
               'EtCO2', 'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST',
               'BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine',
               'Bilirubin_direct', 'Glucose', 'Lactate', 'Magnesium', 'Phosphate',
               'Potassium', 'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
               'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
               'HospAdmTime', 'ICULOS']

    # Reformatting data into DataFrame to add features
    patient_data = pd.DataFrame(data, columns=columns)

    # Handling Missing values
    # patient_data[vital_signs] = patient_data[vital_signs].rolling(window=6, min_periods=1).mean().bfill()
    patient_data = patient_data.bfill()
    patient_data = patient_data.ffill()
    patient_data = patient_data.fillna(0)

    # patient_data['MAP_SOFA'] = patient_data['MAP'].apply(map_sofa)
    patient_data['MAP_SOFA'] = map_sofa(patient_data['MAP'])
    patient_data['Bilirubin_total_SOFA'] = patient_data['Bilirubin_total'].apply(total_bilirubin_sofa)
    patient_data['Platelets_SOFA'] = patient_data['Platelets'].apply(platelets_sofa)
    patient_data['SOFA_score'] = patient_data.apply(sofa_score, axis=1)
    patient_data = detect_sofa_change(patient_data)

    patient_data['ResP_qSOFA'] = patient_data['Resp'].apply(respiratory_rate_qsofa)
    patient_data['SBP_qSOFA'] = patient_data['SBP'].apply(sbp_qsofa)
    patient_data['qSOFA_score'] = patient_data.apply(qsofa_score, axis=1)
    patient_data = detect_qsofa_change(patient_data)

    patient_data['qSOFA_indicator'] = patient_data.apply(q_sofa_indicator, axis=1)  # Sepsis detected
    patient_data['SOFA_indicator'] = patient_data.apply(sofa_indicator, axis=1)  # Organ Dysfunction occurred
    patient_data['Mortality_sofa'] = patient_data.apply(mortality_sofa, axis=1)  # Morality rate

    patient_data['Temp_sirs'] = patient_data['Temp'].apply(temp_sirs)
    patient_data['HR_sirs'] = patient_data['HR'].apply(heart_rate_sirs)
    patient_data['Resp_sirs'] = patient_data['Resp'].apply(resp_sirs)
    patient_data['paco2_sirs'] = patient_data['PaCO2'].apply(resp_sirs)
    patient_data['wbc_sirs'] = patient_data['WBC'].apply(wbc_sirs)

    # patient_data = t_suspicion(patient_data)
    # patient_data = t_sofa(patient_data)
    # patient_data['t_sepsis'] = patient_data.apply(t_sepsis, axis=1)

    # patient_data = patient_data[final_features]

    max_rows = 8
    num_features = patient_data.shape[1]
    if len(patient_data) < max_rows:
        padding = np.zeros((max_rows - len(patient_data), num_features))
        patient_data = np.vstack((patient_data.values, padding)).astype(np.float32)

    else:
        
        # If patient data is > window_size ?
        segments = []
        max_rows = 8

        for start in range(0, len(patient_data) - max_rows + 1):
            segment = patient_data.iloc[start: start + max_rows].values
            segments.append(segment.astype(np.float32))

        # Making predictions for every segment
        segment_predictions = []
        segment_probs = []
        for segment in segments:
            segment = torch.from_numpy(segment).unsqueeze(0)

            model.eval()
            model.to(device)

            with torch.no_grad():
                segment = segment.to(torch.float32).to(device)
                outputs, _, _, _, _, _, _ = model(segment, stage='test')

                _, predicted = torch.max(outputs, 1)
                probabilities = F.softmax(outputs, dim=1)

            predicted_class = predicted.detach().cpu().numpy()[0]
            probabilities = probabilities.detach().cpu().numpy()[0][predicted_class]

            segment_predictions.append(predicted_class)
            segment_probs.append(probabilities)
        
        print(np.mean(segment_predictions), segment_probs)
        predictions = [torch.tensor(segment_predictions).mode()[0].detach().cpu().numpy().item()]
        probas = [torch.tensor(segment_probs).mean().detach().cpu().numpy().item()]
        
        return predictions, probas, patient_data

    # This code executes when patient_data is within the window_size
    patient_data = torch.from_numpy(patient_data).unsqueeze(0)

    model.eval()
    model.to(device)
    predictions = []
    probas = []

    with torch.no_grad():
        patient_data = patient_data.to(torch.float32).to(device)
        outputs, _, _, _, _, _, _ = model(patient_data, stage='test')

        _, predicted = torch.max(outputs, 1)
        probabilities = F.softmax(outputs, dim=1)

        predicted_class = predicted.detach().cpu().numpy()[0]

        predictions.append(predicted_class)
        probas.append(probabilities.detach().cpu().numpy()[0][predicted_class])

    return predictions, probas, patient_data

In [ ]:
sample = pd.read_csv('/localscratch/neeresh/data/physionet2019/physionet.org/files/challenge-2019/1.0.0/training/training_setA/p000034.psv', sep='|')
sample = sample.drop(['SepsisLabel'], axis=1).values

num_rows = len(sample)  # Number of patient recordings
scores = np.zeros(num_rows)
labels = np.zeros(num_rows)

for t in range(num_rows):
    current_data = sample[:t + 1]
    current_labels, current_score, data_df = get_sepsis_score(current_data, model)
    scores[t] = current_score[0]
    labels[t] = current_labels[0]
    

In [ ]:
len(scores), len(labels), sample.shape

In [ ]:
labels